In [1]:
# !pip install textblob
# !pip install gensim
# nltk.download('stopwords')
# python -m spacy download en_core_web_sm

In [2]:
import re
import numpy as np
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords

In [3]:
# Importing the training data

data_folder_path = "C:\\Users\\swapn\\OneDrive\\Desktop\\NLP\\Assignment2\\NLP_AUTUMN_ASSIGNMENT_DATA\\"

train_file_path = data_folder_path + "NLP_ass_train.tsv"
val_file_path = data_folder_path + "NLP_ass_valid.tsv"
test_file_path = data_folder_path + "NLP_ass_test.tsv"

train_df = pd.read_csv(train_file_path, delimiter='\t')
val_df = pd.read_csv(val_file_path, delimiter='\t')
test_df = pd.read_csv(test_file_path, delimiter='\t')


# Renaming the columns
df1 = pd.DataFrame({train_df.columns[0]: [train_df.columns[0]], train_df.columns[1]: [train_df.columns[1]]})
train_df = pd.concat([df1, train_df])
train_df = train_df.reset_index(drop=True)
train_df.columns = ["Text", "Label"]

df2 = pd.DataFrame({val_df.columns[0]: [val_df.columns[0]], val_df.columns[1]: [val_df.columns[1]]})
val_df = pd.concat([df2, val_df])
val_df = val_df.reset_index(drop=True)
val_df.columns = ["Text", "Label"]

df3 = pd.DataFrame({test_df.columns[0]: [test_df.columns[0]], test_df.columns[1]: [test_df.columns[1]]})
test_df = pd.concat([df3, test_df])
test_df = test_df.reset_index(drop=True)
test_df.columns = ["Text", "Label"]

In [4]:
train_df

,Text,Label
0,we cannot continue calling ourselves feminists...,normal
1,nawt yall niggers ignoring me,normal
2,<user> i am bit confused coz chinese ppl can n...,hatespeech
3,and this is why i end up with nigger trainee d...,hatespeech
4,nogs jews and dykes how enriching,offensive
...,...,...
15378,this vile racist ghetto ass dumber than a reta...,offensive
15379,when i first got on here and said i hate trump...,offensive
15380,was macht der moslem wenn der zion gegen seine...,normal
15381,it is awful look at world demographics asians ...,hatespeech


In [5]:
val_df

,Text,Label
0,this bitch in whataburger eating a burger with...,hatespeech
1,tired i can not support abortion from a moral ...,hatespeech
2,all of my exes were cute but they were hoes i ...,offensive
3,<user> <user> they visited helped provided foo...,normal
4,it funny how people offended by the word nigge...,normal
...,...,...
1917,why do you think all these various muslim grou...,offensive
1918,this is pretty much the conclusion of the fran...,hatespeech
1919,graph straftaten gegen die sexuelle selbstbest...,normal
1920,an afghani immigrant once told me that in afgh...,normal


In [6]:
test_df

,Text,Label
0,i dont think im getting my baby them white 9 h...,normal
1,laura loomer raped me while screaming at me in...,hatespeech
2,<user> what did the old lady do was she a nazi...,normal
3,as much as i appreciate a format of open discu...,normal
4,sex be so good a bitch be slow stroking and cr...,offensive
...,...,...
1919,it always women trying this shit like dingbat ...,offensive
1920,because women would never lie about being sexu...,offensive
1921,how is all that awesome muslim diversity going...,offensive
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [7]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters and symbols
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Handling Negations
    # Detect and handle negations by appending "not_" to the words that follow negation terms.
    words = text.split()
    for i in range(len(words)):
        if words[i] == 'not' and i < len(words) - 1:
            words[i + 1] = 'not_' + words[i + 1]
    
    text = ' '.join(words)
    
    # Tokenize the text (split it into words)
    words = text.split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)
    
    # Token Normalization
    # Normalize words by reducing them to their base forms.
    words = text.split()
    normalized_words = [Word(word).lemmatize() for word in words]
    
    # Join the words back into a cleaned text
    cleaned_text = ' '.join(normalized_words)
    return cleaned_text


In [8]:
# Preprocessing the data
train_df['Text'] = train_df['Text'].apply(preprocess_text)
val_df['Text'] = val_df['Text'].apply(preprocess_text)
test_df['Text'] = test_df['Text'].apply(preprocess_text)

In [9]:
train_df

,Text,Label
0,cannot continue calling feminist right womxn a...,normal
1,nawt yall nigger ignoring,normal
2,user bit confused coz chinese ppl not_access t...,hatespeech
3,end nigger trainee doctor not_speak properly l...,hatespeech
4,nog jew dyke enriching,offensive
...,...,...
15378,vile racist ghetto as dumber retarded tree sna...,offensive
15379,first got said hate trump considered troll peo...,offensive
15380,macht der moslem wenn der zion gegen seinen pr...,normal
15381,awful look world demographic asian fucking eve...,hatespeech


In [10]:
val_df

,Text,Label
0,bitch whataburger eating burger top bun holdin...,hatespeech
1,tired not_support abortion moral standpoint wi...,hatespeech
2,ex cute hoe guess attract fly looking thots,offensive
3,user user visited helped provided food kp isla...,normal
4,funny people offended word nigger problem call...,normal
...,...,...
1917,think various muslim group middle east india a...,offensive
1918,pretty much conclusion frankfurt school said o...,hatespeech
1919,graph straftaten gegen die sexuelle selbstbest...,normal
1920,afghani immigrant told afghanistan not_believe...,normal


In [11]:
test_df

,Text,Label
0,dont think im getting baby white 9 two white j...,normal
1,laura loomer raped screaming disgusting kike l...,hatespeech
2,user old lady nazi white supremacist,normal
3,much appreciate format open discussion really ...,normal
4,sex good bitch slow stroking cry,offensive
...,...,...
1919,always woman trying shit like dingbat dingell ...,offensive
1920,woman would never lie sexually assaulted perso...,offensive
1921,awesome muslim diversity going native german a...,offensive
1922,well dear lgbtq brother sister not_want die ha...,hatespeech


In [12]:
from sklearn.preprocessing import LabelEncoder

X_train = train_df[train_df.columns[0]]
X_val = val_df[val_df.columns[0]]
X_test = test_df[test_df.columns[0]]
y_train = train_df[train_df.columns[1]]
y_val = val_df[val_df.columns[1]]
y_test = test_df[test_df.columns[1]]

# Encode the labels using LabelEncoder
# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_val = label_encoder.fit_transform(y_val)
# y_test = label_encoder.fit_transform(y_test)

In [13]:
y_train

0            normal
1            normal
2        hatespeech
3        hatespeech
4         offensive
            ...    
15378     offensive
15379     offensive
15380        normal
15381    hatespeech
15382     offensive
Name: Label, Length: 15383, dtype: object

In [14]:
import pandas as pd
import spacy

# Load your DataFrame
# Assuming 'df' is your DataFrame with a 'Comment Text' column

# Extract sentences from the DataFrame
train_sentences = X_train.tolist()
val_sentences = X_val.tolist()
test_sentences = X_test.tolist()

# Tokenize the sentences using spaCy
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
train_tokenized_sentences = [[token.text for token in nlp(sentence)] for sentence in train_sentences]
val_tokenized_sentences = [[token.text for token in nlp(sentence)] for sentence in val_sentences]
test_tokenized_sentences = [[token.text for token in nlp(sentence)] for sentence in test_sentences]

# Flatten the tokens and remove duplicates
all_tokens = [token for sentence_tokens in train_tokenized_sentences for token in sentence_tokens]
unique_tokens = list(set(all_tokens))

# Add special tokens if needed
special_tokens = ['<PAD>', '<UNK>', '<START>', '<END>']
unique_tokens = special_tokens + unique_tokens

# Create a token-to-index dictionary
token_to_index = {token: index for index, token in enumerate(unique_tokens)}
index_to_token = {index: token for index, token in enumerate(unique_tokens)}

# Vocabulary size
vocab_size = len(unique_tokens)


In [15]:
# unique_tokens
val_tokenized_sentences

[['bitch',
  'whataburger',
  'eating',
  'burger',
  'top',
  'bun',
  'holding',
  'palm',
  'hate',
  'white',
  'bitch'],
 ['tired',
  'not_support',
  'abortion',
  'moral',
  'standpoint',
  'wired',
  'support',
  'morally',
  'easy',
  'stop',
  'looking',
  'little',
  'niglets',
  'future',
  'doctor',
  'see',
  'really',
  'future',
  'rapist',
  'white',
  'woman',
  'future',
  'welfare',
  'recipient',
  'white',
  'tax',
  'dollar',
  'future',
  'democrat',
  'voter'],
 ['ex', 'cute', 'hoe', 'guess', 'attract', 'fly', 'looking', 'thots'],
 ['user',
  'user',
  'visited',
  'helped',
  'provided',
  'food',
  'kp',
  'islam',
  'c',
  'terrorist',
  'butured',
  'raped',
  'forced',
  'leave',
  'land'],
 ['funny',
  'people',
  'offended',
  'word',
  'nigger',
  'problem',
  'calling',
  'people',
  'nazi',
  'difference'],
 ['photo',
  'look',
  'rhyan',
  'moody',
  'killed',
  'thanksgiving',
  'illegal',
  'spic'],
 ['user',
  'user',
  'shoe',
  'foot',
  'would'

In [16]:
max_sequence_length = max(len(sentence_tokens) for sentence_tokens in train_tokenized_sentences)

In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert the tokenized sentences to sequences of indices
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokenized_sentences)
train_sequences = tokenizer.texts_to_sequences(train_tokenized_sentences)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(val_tokenized_sentences)
val_sequences = tokenizer.texts_to_sequences(val_tokenized_sentences)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_tokenized_sentences)
test_sequences = tokenizer.texts_to_sequences(test_tokenized_sentences)

# Pad or truncate the sequences to the specified maximum sequence length
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# 'sequences' now contains your padded or truncated data.


In [18]:
val_sequences

array([[   7, 2811, 1281, ...,    0,    0,    0],
       [ 357, 2815,  574, ...,    0,    0,    0],
       [2820, 2821,   85, ...,    0,    0,    0],
       ...,
       [2810, 6760, 6761, ...,    0,    0,    0],
       [1932,   31,  204, ...,    0,    0,    0],
       [ 340,   73,  141, ...,    0,    0,    0]])

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.models import Sequential

class TextClassificationRNN(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, num_classes, max_sequence_length, dropout_rate):
        super(TextClassificationRNN, self).__init__()  # Corrected super statement
        
        # Embedding Layer
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)
        
        # RNN Layer (SimpleRNN, you can use other RNN variants like LSTM or GRU)
        self.rnn = SimpleRNN(units=rnn_units)
        
        # Fully connected classification layer
        self.fc = Dense(units=num_classes, activation='softmax')
        
        # Dropout layer for regularization
        self.dropout = Dropout(rate=dropout_rate)
    
    def call(self, inputs):
        # Forward pass
        x = self.embedding(inputs)
        x = self.rnn(x)
        x = self.fc(x)
        return x



In [20]:
# Define model hyperparameters
vocab_size = len(unique_tokens) # Replace with the actual vocabulary size
embedding_dim = 300  # Choose an appropriate dimension
rnn_units = 64 # Choose the number of RNN units
num_classes = 3 # Set the number of output classes
dropout_rate = 0.5  # Choose the dropout rate for regularization

# Create an instance of the model
model = TextClassificationRNN(vocab_size, embedding_dim, rnn_units, num_classes, max_sequence_length, dropout_rate)

# Compile the model with appropriate loss, optimizer, and metrics
model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
import pandas as pd

# Assuming y_train and y_val are Pandas Series with string labels
label_to_int = {'normal': 0, 'hatespeech': 1, 'offensive': 2}

# Map the string labels to integers for y_train and y_val
y_train = y_train.map(label_to_int)
y_val = y_val.map(label_to_int)
y_test = y_test.map(label_to_int)

# Convert the mapped labels to int32
y_train = y_train.astype('int32')
y_val = y_val.astype('int32')
y_test = y_test.astype('int32')

# One-hot encode the labels
num_classes = 3  # Replace with the actual number of classes
y_train = pd.get_dummies(y_train)
y_val = pd.get_dummies(y_val)
y_test = pd.get_dummies(y_test)

In [22]:
# Assuming you have preprocessed training and validation data and labels in numpy arrays
X_train = train_sequences # Training data (tokenized and padded/truncated sequences)
y_train = y_train  # Training labels (one-hot encoded)
X_val = val_sequences  # Validation data
y_val = y_val    # Validation labels
X_test = test_sequences  # Test data
y_test = y_test    # Test labels


In [23]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)


In [24]:
# Specify the number of training epochs
epochs = 100  # Replace with the desired number of training epochs
batch_size = 64

# Train the model with a separate validation set with early stoping
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/100
481/481 [==============================] - 77s 157ms/step - loss: 1.0542 - accuracy: 0.4573 - val_loss: 1.1650 - val_accuracy: 0.3215
Epoch 2/100
481/481 [==============================] - 67s 140ms/step - loss: 0.9736 - accuracy: 0.5362 - val_loss: 1.1146 - val_accuracy: 0.4360
Epoch 3/100
481/481 [==============================] - 78s 161ms/step - loss: 0.9680 - accuracy: 0.5402 - val_loss: 1.1204 - val_accuracy: 0.4142
Epoch 4/100
445/481 [==========================>...] - ETA: 5s - loss: 0.9712 - accuracy: 0.5395

KeyboardInterrupt: 

In [ ]:
# Save the model in TensorFlow SavedModel format
model.save('RNN_Model', save_format='tf')

INFO:tensorflow:Assets written to: RNN_Model\assets


INFO:tensorflow:Assets written to: RNN_Model\assets


In [ ]:
# Save the model weights
model.save_weights('RNN_Model_Weights.h5')

# Save the model architecture (including configuration)
with open('RNN_Model_Architecture.json', 'w') as json_file:
    json_file.write(model.to_json())

In [ ]:
import tensorflow as tf

# Load the SavedModel
loaded_model = tf.keras.models.load_model('RNN_Model')

In [ ]:
# Assuming X_test is your test data
predictions = loaded_model.predict(X_test)

# The 'predictions' variable will contain the model's predictions on your test data
from sklearn.metrics import accuracy_score

# Assuming y_test is your true labels
true_labels = y_test.idxmax(axis=1).to_numpy().astype('int64')

# Convert the model's predictions to class labels (e.g., argmax for classification)
predicted_labels = tf.argmax(predictions, axis=1).numpy()

predicted_labels


61/61 [==============================] - 1s 8ms/step


array([0, 2, 0, ..., 0, 2, 2], dtype=int64)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {100*accuracy:.4f}%')

Accuracy: 39.3971%


In [ ]:
from sklearn.metrics import f1_score
# Calculate the macro F1 score
macro_f1 = f1_score(true_labels, predicted_labels, average='macro')

print("Macro F1 Score:", macro_f1)

In [ ]:
# Test Accuracy: 39.3971%

In [ ]:
def calculate_intersection(set1, set2):
    return len(set(set1) & set(set2))

# Example train, validation, and test sets (replace with your actual data)
train_set = set(train_df["Text"])
validation_set = set(val_df["Text"])
test_set = set(test_df["Text"])

# Calculate and print the intersections
intersection_train_validation = calculate_intersection(train_set, validation_set)
intersection_train_test = calculate_intersection(train_set, test_set)
intersection_validation_test = calculate_intersection(validation_set, test_set)

print("Intersection between train and validation sets:", intersection_train_validation)
print("Intersection between train and test sets:", intersection_train_test)
print("Intersection between validation and test sets:", intersection_validation_test)